In [9]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from numpy import argmax, array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
from sklearn.metrics import mean_squared_error
import math

In [4]:
train = pd.read_csv("kdd_train.csv")
test = pd.read_csv("kdd_test.csv")

print(test.shape,train.shape)
#print(test['duration'])

(22544, 42) (125973, 42)


In [5]:
for i in ['protocol_type', 'service', "flag", "labels"]:
    values_train = array(train[i])
    values_test = array(test[i])
    label_encoder = LabelEncoder()
    integer_encoded_train = label_encoder.fit_transform(values_train)
    integer_encoded_test = label_encoder.fit_transform(values_test)
    train[i] = integer_encoded_train
    test[i] = integer_encoded_test

X_train = train.iloc[:, :41]
y_train = train.iloc[:, 41]

X_test = test.iloc[:, :41]
y_test = test.iloc[:, 41]

#print(X_train.head(),y_train.head())
print(X_test.head(),y_test.head())
#print(list(y_train))

   duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0         5              1       51     9       2429        475     0   
1         0              2       12     9         45        134     0   
2         0              2       12     9         45         80     0   
3      1979              2       41     9        145        105     0   
4     14462              1       41     4          1          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_count  dst_host_srv_count  \
0               0       0    0  ...             180                  57   
1               0       0    0  ...             194                 155   
2               0       0    0  ...             255                 255   
3               0       0    0  ...             255                   2   
4               0       0    0  ...             255                   2   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.11                    0.02   
1   

In [6]:

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=20)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
rfe_X_train = pd.DataFrame()
rfe_X_test = pd.DataFrame()
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    col_name_train = train[train.columns[i]]
    col_name_test = test[test.columns[i]]
    if rfe.support_[i]:
        rfe_X_train = pd.concat([rfe_X_train, col_name_train], axis = 1)
        rfe_X_test = pd.concat([rfe_X_test, col_name_test], axis = 1)
      
print (rfe_X_train.shape, rfe_X_test.head())
print(col_name_train)


Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected False, Rank: 2.000
Column: 6, Selected False, Rank: 5.000
Column: 7, Selected True, Rank: 1.000
Column: 8, Selected False, Rank: 13.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 3.000
Column: 11, Selected False, Rank: 7.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 12.000
Column: 14, Selected False, Rank: 17.000
Column: 15, Selected False, Rank: 16.000
Column: 16, Selected False, Rank: 10.000
Column: 17, Selected False, Rank: 9.000
Column: 18, Selected False, Rank: 18.000
Column: 19, Selected False, Rank: 21.000
Column: 20, Selected False, Rank: 22.000
Column: 21, Selected False, Rank: 20.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 8.000
Column: 24, Selected True, Rank: 1.000
Column: 25,

In [23]:
#ELM
input_size = rfe_X_train.shape[1]
hidden_size = 20

input_weights = np.random.uniform(-5,5,size=(input_size,hidden_size))
#input_weights = np.random.normal(size=[input_size, hidden_size])
biases = np.random.normal(size=[hidden_size])

print(input_weights,biases)


[[-1.11866772e+00  2.39591680e+00  1.90674268e+00 -2.67134321e+00
   1.61637887e+00 -4.97268781e-03  3.68078673e+00 -1.12266572e+00
  -1.11270234e+00  3.51318275e+00 -4.74441318e+00 -4.52919675e+00
   1.79631717e+00 -1.92622057e+00  4.28109503e+00 -5.96042903e-01
   4.41486771e+00 -9.57216164e-01 -4.31279664e+00  1.10666250e+00]
 [-4.85367969e-01  6.69500262e-01  4.21540121e+00  3.73197409e+00
  -2.85517190e+00 -1.26723181e+00 -6.96723576e-01 -4.60962469e+00
  -9.38825074e-01 -1.09123875e+00  1.01402456e+00  4.94283036e+00
   3.83096848e+00 -2.53633104e+00  1.50504099e+00  2.31635738e+00
  -5.24737558e-01 -1.36475398e-01  4.23566606e-01  2.44658966e+00]
 [ 4.84083474e-01 -1.68729465e+00 -2.96077752e+00  3.28662148e-01
  -2.16884611e+00  1.52390632e+00 -2.10404727e+00 -4.71852022e+00
  -2.26125020e+00 -3.38804347e+00  1.97434252e+00  4.72189389e+00
  -4.47497809e+00  3.65479188e+00 -3.31106097e+00 -2.72430366e+00
   4.77612666e+00  1.14345678e+00 -3.34522365e+00 -1.65355624e-01]
 [-1.47

In [24]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = sigmoid(G)
    return H

output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train)), y_train)

<ipython-input-24-1ed99a0c591d>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [25]:

def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(rfe_X_test)
print("MSE: ",mean_squared_error(y_test,prediction))
print("RMSE: ",math.sqrt(mean_squared_error(y_test,prediction)))

MSE:  47.31095358876977
RMSE:  6.878295834635915
<ipython-input-24-1ed99a0c591d>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [12]:

correct = 0
total = rfe_X_test.shape[0]
sum1 = 0

for i in range(total):
    predicted = prediction[i]
    actual = y_test[i]
    #print(prediction[i], predicted, y[i], actual)
    sum1+= ((actual - predicted)**2)
    #print(sum1)

mse = sum1/total

print('Accuracy for ', hidden_size, ' hidden nodes: ', mse)

Accuracy for  20  hidden nodes:  47.35212458537501
